# Option 1: Crawl news & information websites & anticipate the likelihood of its virality. 
## By Arpan Ghoshal

### My approach is to evaluate similarity between the articles. As the viral news should be spread throught all the news pages, more it is similar, more viral it will be.
### My target website is BBC/world and reference is TOI/world. 

In [ ]:
import requests
from bs4 import BeautifulSoup
from newspaper import Article  
import csv 
import pandas as pd
import numpy as np

In [2]:
response = requests.get('http://www.bbc.co.uk/news/world')
doc = BeautifulSoup(response.text, 'html.parser')

In [4]:
bbc_list=[]
headlines = doc.find_all('h3')

for headline in headlines:
    bbc_list.append(headline.text)
print(bbc_list)

["Italy moves to ease Europe's longest lockdown", "Italy moves to ease Europe's longest lockdown", 'Nations relax restrictions as new cases fall', 'Nations relax restrictions as new cases fall', "Spain's children allowed to run free again", 'Why world will look to India to make Covid-19 vaccine', "Chile to introduce controversial 'virus-free' ID", 'Saudi commission announces change to death penalty', 'Yemen separatists declare self-rule in south', "Italy moves to ease Europe's longest lockdown", 'Nations relax restrictions as new cases fall', 'Nations relax restrictions as new cases fall', "Spain's children allowed to run free again", 'Why world will look to India to make Covid-19 vaccine', "Chile to introduce controversial 'virus-free' ID", 'Saudi commission announces change to death penalty', 'Yemen separatists declare self-rule in south', 'Nations relax restrictions as new cases fall', 'Nations relax restrictions as new cases fall', "Spain's children allowed to run free again", 'Why

In [7]:
r = requests.get("https://timesofindia.indiatimes.com/world")
soup = BeautifulSoup(r.content, 'html5lib') 
table = soup.findAll('a', attrs = {'class':'w_img'}) 
news=[]
for row in table: 
    if not row['href'].startswith('http'):
        news.append('https://timesofindia.indiatimes.com'+row['href'])
dfnakli=[]
for i in news:
    article = Article(i, language="en")
    article.download() 
    article.parse() 
    article.nlp() 
    data={}
    data['Title']=article.title
    dfnakli.append(data)

In [13]:
dataset=pd.DataFrame(dfnakli)
title = dataset["Title"].values
toi_list=title.tolist()
toi_list

['Saudi Arabia ends death penalty for minors and floggings',
 'Covid-19: Lockdown reveals fresh and clean air in India',
 "Iran's Guard says it launched satellite",
 'Covid-19: Virus shutdown shows human impact on Earth',
 'San Francisco: Dog stolen in December has joyous reunion with owner',
 'Crude Oil prices drop to historic lows after extreme collapse in demand',
 'Covid-19: Los Angeles study indicates higher infection rate',
 "Covid-19: President Trump says 'politics' is behind testing criticism",
 'Covid-19: Cemetery races to keep up as New York deaths mount',
 "Texans visit state parks in governor’s 'reopening'",
 'Covid-19: Protest in favour of easing lockdown in US',
 'Amid virus crisis, heart and stroke patients go missing',
 'coronavirus: Pak denies preparing vaccine for COVID-19; cases cross 12,000',
 'Bangladesh garment factories reopen, defying virus lockdown',
 'Covid-19: Kolkata woman in Oxford University vaccine project team',
 'As virus lockdown eases, Italy ponders w

In [14]:
import nltk, string
from sklearn.feature_extraction.text import TfidfVectorizer


stemmer = nltk.stem.porter.PorterStemmer()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)

def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

def normalize(text):
    return stem_tokens(nltk.word_tokenize(text.lower().translate(remove_punctuation_map)))

vectorizer = TfidfVectorizer(tokenizer=normalize, stop_words='english')

def cosine_sim(text1, text2):
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]

In [31]:
stem_tokens(bbc_list)
stem_tokens(toi_list)

In [20]:
dic1={}
n=0
for i in range(len(bbc_list)):
    for j in range(len(toi_list)):
        dic1[(cosine_sim(bbc_list[i-1],toi_list[j-1]))]=bbc_list[i-1]

In [22]:
df = pd.DataFrame(list(dic1.items()),columns = ['Similarity','News']) 

In [26]:
df=df.groupby(['News']).mean()

In [29]:
Final = df.sort_values(['Similarity'], ascending=[False])
Final

,Similarity
News,
Saudi commission announces change to death penalty,0.304126
India's circuses struggle to survive the lockdown,0.206137
Italy moves to ease Europe's longest lockdown,0.170218
Why so much US resistance to the lockdown?,0.133679
"In China, finding hope amid coronavirus",0.118326
Could coronavirus redefine heroism?,0.117486
Can these robot roommates liven up lockdown?,0.113020
Why do African states differ over lockdowns?,0.112343
Why world will look to India to make Covid-19 vaccine,0.111267


### As seen the most similar are most likely to be viral